In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import timeit
import datetime
import time
import pprint
import itertools
import pickle
import sklearn
import dask
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
import fastparquet
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import h5py
import dask
import dask.dataframe as dd
import dask.array as da
# Percentiles
from scipy.stats import percentileofscore
from IPython.display import display

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Compare new assignees at new location's similarity to assignees who already cite prior patent

In [94]:
yv = "appyear"
c2 = pd.read_pickle("DataStore/2018-08/inv_move_cites_0912.pkl")
ip = pd.read_pickle("DataStore/2018-08/inv_move_pats_0912.pkl")

# Use unique assignees
c2 = c2.drop(["cited_asg", "citing_asg"],1)
asgs = fastparquet.ParquetFile("RawData/Cleaned/patent_assignees_unique_0628.parq").to_pandas(["patent", "assignee_id"])
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq")\
.to_pandas(["patent", "primclass"])
pdf = pdf.merge(asgs, how = "left", on = "patent")

c2 = c2.merge(pdf.add_prefix("citing_"), how="left", left_on="citing", right_on="citing_patent").\
drop("citing_patent",1).rename(columns={"citing_assignee_id": "citing_asg"})
c2 = c2.merge(pdf.add_prefix("cited_"), how="left", left_on="cited", right_on="cited_patent").\
drop("cited_patent",1).rename(columns={"cited_assignee_id": "cited_asg"})
del(asgs)

# New firms that cite prior patent post move
a1 = c2.loc[(c2["citing_appyear"] < c2["sec_fyear"]), "citing_asg"].tolist()
a2 = c2.loc[(c2["citing_appyear"] >= c2["sec_fyear"]), "citing_asg"].tolist()
new_cite_asgs = list(set(a2).difference(set(a1)))
prev_cite_asgs = list(set(a2).intersection(set(a1)))

In [14]:
# Patents by newly citing assignees
pdf = pdf.loc[pdf["assignee_id"].isin(new_cite_asgs)]
len(pdf)

# Sort by assignee, primclass, app year
pdf = pdf.sort_values(["assignee_id", "primclass", yv], ascending = [1,1,0])

# Patents in asg, PC 5 years before and after a yr
# Patents in asg 5 years before and after a yr
asg_pc_prior = {}
asg_pc_post = {}
asg_prior = {}
asg_post = {}

for yr in range(1975, 2016):
    print(yr)
    print(datetime.datetime.now())
    prior_df = pdf.loc[pdf["appyear"].isin(range(yr-5,yr))]
    post_df = pdf.loc[pdf["appyear"].isin(range(yr,yr+5))]
    
    # Control patent by assignee, primclass
    prior = prior_df.groupby(["assignee_id", "primclass"])
    prior = {n+(yr,): (g["patent"].tolist() if len(g["patent"].tolist()) >= 1 else np.nan) for n,g in prior}
    asg_pc_prior.update(prior)
    del(prior)
    
    # Control patent by assignee
    post = post_df.groupby(["assignee_id", "primclass"])
    post = {n+(yr,): (g["patent"].tolist() if len(g["patent"].tolist()) >= 1 else np.nan) for n,g in post}
    asg_pc_post.update(post)
    del(post)
    
    # Control patent by assignee, primclass
    prior = prior_df.groupby(["assignee_id"])
    prior = {(n,yr): (g["patent"].tolist() if len(g["patent"].tolist()) >= 1 else np.nan) for n,g in prior}
    asg_prior.update(prior)
    del(prior)
    
    # Control patent by assignee
    post = post_df.groupby(["assignee_id"])
    post = {(n,yr): (g["patent"].tolist() if len(g["patent"].tolist()) >= 1 else np.nan) for n,g in post}
    asg_post.update(post)
    del(post)

1975
2018-09-17 12:02:25.436338
1976
2018-09-17 12:02:27.076735
1977
2018-09-17 12:02:29.034630
1978
2018-09-17 12:02:31.211717
1979
2018-09-17 12:02:33.598309
1980
2018-09-17 12:02:36.351060
1981
2018-09-17 12:02:39.166655
1982
2018-09-17 12:02:42.421744
1983
2018-09-17 12:02:45.865612
1984
2018-09-17 12:02:49.176962
1985
2018-09-17 12:02:52.770942
1986
2018-09-17 12:02:56.489600
1987
2018-09-17 12:03:00.506504
1988
2018-09-17 12:03:04.918343
1989
2018-09-17 12:03:09.754816
1990
2018-09-17 12:03:14.991635
1991
2018-09-17 12:03:20.812333
1992
2018-09-17 12:03:26.742918
1993
2018-09-17 12:03:33.342392
1994
2018-09-17 12:03:40.512689
1995
2018-09-17 12:03:49.871923
1996
2018-09-17 12:04:00.868284
1997
2018-09-17 12:04:11.860944
1998
2018-09-17 12:04:23.829267
1999
2018-09-17 12:04:36.155048
2000
2018-09-17 12:04:49.999871
2001
2018-09-17 12:05:03.773307
2002
2018-09-17 12:05:18.730815
2003
2018-09-17 12:05:33.380874
2004
2018-09-17 12:05:48.450422
2005
2018-09-17 12:06:02.753138
2006
201

In [96]:
# Get values
# New cites
c3 = c2.loc[(c2["citing_appyear"] >= c2["sec_fyear"]) & c2["citing_asg"].isin(new_cite_asgs)]

# Merge new citing patents with pdf
c3["citing_primclass"] = c3["citing"].map(dict(zip(pdf["patent"], pdf["primclass"])))

c3["citing_asg_pc_prior"] = [asg_pc_prior.get((asg, pc, fyr)) for asg,pc,fyr in \
                        zip(c3["citing_asg"], c3["citing_primclass"], c3["sec_fyear"])]
c3["citing_asg_pc_post"] = [asg_pc_post.get((asg, pc, fyr)) for asg,pc,fyr in \
                        zip(c3["citing_asg"], c3["citing_primclass"], c3["sec_fyear"])]
c3["citing_asg_cited_pc_prior"] = [asg_pc_prior.get((asg, pc, fyr)) for asg,pc,fyr in \
                        zip(c3["citing_asg"], c3["cited_primclass"], c3["sec_fyear"])]
c3["citing_asg_cited_pc_post"] = [asg_pc_post.get((asg, pc, fyr)) for asg,pc,fyr in \
                        zip(c3["citing_asg"], c3["cited_primclass"], c3["sec_fyear"])]
c3["citing_asg_prior"] = [asg_prior.get((asg, fyr)) for asg,fyr in \
                        zip(c3["citing_asg"], c3["sec_fyear"])]
c3["citing_asg_post"] = [asg_post.get((asg, fyr)) for asg,fyr in \
                        zip(c3["citing_asg"], c3["sec_fyear"])]

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [35]:
# # If I want to remove the citing patent in all cases
# def get_pat_list_asg_pc(d, asg, pc, fyr, c):
#     # Get list
#     l1 = d.get((asg, pc, fyr))
#     # Return none
#     if not l1:
#         return l1
#     else:
#     # Remove c if it's in the list
#         if (c in l1):
#             l1.remove(c)
#     return l1

# def get_pat_list_asg(d, asg, fyr, c):
#     # Get list
#     l1 = d.get((asg, fyr))
#     # Return none
#     if not l1:
#         return l1
#     else:
#     # Remove c if it's in the list
#         if (c in l1):
#             l1.remove(c)
#     return l1
        

# # Get values
# %time c3["citing_asg_pc_prior"] = [get_pat_list_asg_pc(asg_pc_prior, asg, pc, fyr, c) for asg,pc,fyr,c in \
#                         zip(c3["citing_asg"], c3["citing_primclass"], c3["sec_fyear"], c3["citing"])]
# %time c3["citing_asg_pc_post"] = [get_pat_list_asg_pc(asg_pc_post, asg, pc, fyr, c) for asg,pc,fyr,c in \
#                         zip(c3["citing_asg"], c3["citing_primclass"], c3["sec_fyear"], c3["citing"])]
# %time c3["citing_asg_prior"] = [get_pat_list_asg(asg_prior, asg, fyr, c) for asg,fyr,c in \
#                         zip(c3["citing_asg"], c3["sec_fyear"], c3["citing"])]
# %time c3["citing_asg_post"] = [get_pat_list_asg(asg_post, asg, fyr, c) for asg,fyr,c in \
#                         zip(c3["citing_asg"], c3["sec_fyear"], c3["citing"])]

In [97]:
# Replace empty lists with None
c3 = c3.mask(c3.applymap(str).eq('[]'))

# Get rid of missing cited-citing values
print(len(c3))
c3 = c3.loc[c3["sim_docvecs"].notnull()]
print(len(c3))

# Get rid of missing all assignee values
c3 = c3.dropna(subset=["citing_asg_pc_prior", "citing_asg_pc_post", "citing_asg_prior",\
                       "citing_asg_post"], how = "all")
print(len(c3))

27817
20797
11140


### Get average similarity to newly citing assignees, before and after move

In [100]:
def grouper(n, iterable):
    """
    >>> list(grouper(3, 'ABCDEFG'))
    [['A', 'B', 'C'], ['D', 'E', 'F'], ['G']]
    """
    iterable = iter(iterable)
    return iter(lambda: list(itertools.islice(iterable, n)), [])


import scipy.spatial.distance as distance
dms = ["ldavecs", "docvecs"]

print("Getting row values")
print(datetime.datetime.now())
pat_dict = fastparquet.ParquetFile("RawData/Cleaned/patabs7615_us_no_dup.parq").to_pandas(["patent"])["patent"].tolist()
pat_dict = dict(zip(pat_dict, range(len(pat_dict))))


l2 = c3[['citing', 'cited',]]

for dm in dms:
    print((dm,"started"))
    print("Loading matrix and dict")
    print(datetime.datetime.now())
    
    if dm == "ldavecs":
        ncols = 60
    else:
        ncols = 100

    pm = fastparquet.ParquetFile("DataStore/2018-07-P2/ML/{0}_pats_0712.parq".format(dm))\
    .to_pandas().values
    
    
    for col in ['citing_asg_pc_prior', 'citing_asg_pc_post', 'citing_asg_cited_pc_prior', 'citing_asg_cited_pc_post',
       'citing_asg_prior', 'citing_asg_post']:
        print("Getting patent pair cosine similarity")
        print(datetime.datetime.now())
        print(col)
        
        l3 = pd.DataFrame({"citing": c3["citing"], "cited": c3["cited"],
            "tp": [[i] for i in c3["cited"].tolist()], "op": c3[col]})
        print(len(l3))
        l3 = l3.loc[l3["citing"].notnull() & l3["cited"].notnull() & l3["op"].notnull()]
        l3 = l3.loc[(l3["tp"].apply(len) >=1) & (l3["op"].apply(len) >=1)]
        # Get row values for each if they exist
        l3["tp_rows"] = [[pat_dict[p] for p in l if p in pat_dict.keys()] for l in l3["tp"].tolist()]
        l3["op_rows"] = [[pat_dict[p] for p in l if p in pat_dict.keys()] for l in l3["op"].tolist()]
        l3 = l3.loc[(l3["tp_rows"].apply(len) >=1) & (l3["op_rows"].apply(len) >=1)]
        print(len(l3))
        
        # Get mean similarity of target to other patent rows
        cos_dis = np.array([np.mean(distance.cdist(pm[i], pm[j], metric="cosine")) for i,j in\
                                           zip(l3["tp_rows"], l3["op_rows"])])

        l3["sim_{0}_{1}".format(dm,col)] = 1-cos_dis
        l3 = l3[["cited", "citing", "sim_{0}_{1}".format(dm,col)]].drop_duplicates()

        # Rename columns
        l2 = l2.merge(l3[["cited", "citing", "sim_{0}_{1}".format(dm,col)]], how = "left", on = ["cited", "citing"])
        l2 = l2.drop_duplicates()
        print(len(l2))
        del(l3)
        print("finished")
        print(datetime.datetime.now())
    del(pm)

Getting row values
2018-09-17 14:21:36.172212
('ldavecs', 'started')
Loading matrix and dict
2018-09-17 14:21:38.473624
Getting patent pair cosine similarity
2018-09-17 14:21:48.152717
citing_asg_pc_prior
11140
2404
10958
finished
2018-09-17 14:21:49.000126
Getting patent pair cosine similarity
2018-09-17 14:21:49.000203
citing_asg_pc_post
11140
5843
10958
finished
2018-09-17 14:21:50.021042
Getting patent pair cosine similarity
2018-09-17 14:21:50.021108
citing_asg_cited_pc_prior
11140
2029
10958
finished
2018-09-17 14:21:50.614452
Getting patent pair cosine similarity
2018-09-17 14:21:50.614515
citing_asg_cited_pc_post
11140
4869
10958
finished
2018-09-17 14:21:51.551202
Getting patent pair cosine similarity
2018-09-17 14:21:51.551294
citing_asg_prior
11140
5926
10958
finished
2018-09-17 14:21:52.791540
Getting patent pair cosine similarity
2018-09-17 14:21:52.791629
citing_asg_post
11140
10375
10958
finished
2018-09-17 14:21:54.816031
('docvecs', 'started')
Loading matrix and dict
2

In [102]:
print(len(c3))
c3 = c3.merge(l2, how="left", on=["cited","citing"])
print(len(c3))

11140
11140


In [103]:
c3.to_pickle("DataStore/2018-08/post_move_new_asgs_cites_0916.pkl")

In [79]:
c3.columns

Index(['citing', 'cited', 'sim_ldavecs', 'sim_docvecs', 'cited_inv_msa',
       'citing_inv_msa', 'cited_appyear', 'citing_appyear', 'sec_inv_msa',
       'sec_fyear', 'sec_inv_msa_match', 'citing_asg', 'cited_asg',
       'citing_primclass', 'citing_asg_pc_prior', 'citing_asg_pc_post',
       'citing_asg_prior', 'citing_asg_post',
       'sim_ldavecs_citing_asg_pc_prior', 'sim_ldavecs_citing_asg_pc_post',
       'sim_ldavecs_citing_asg_prior', 'sim_ldavecs_citing_asg_post',
       'sim_docvecs_citing_asg_pc_prior', 'sim_docvecs_citing_asg_pc_post',
       'sim_docvecs_citing_asg_prior', 'sim_docvecs_citing_asg_post'],
      dtype='object')

### Analysis
- Firms that newly started citing the inventor after move to the same MSA
- Measured the average similarity of:
    1. Firm's 5 yr previous patents to new inventor's move
    2. Firm's 5 yr post patents
    3. Firm's 5 yr previous patent matching the cited patent's primary class
    4. Firm's 5 yr post patent matching the cited patent's primary class
    5. Firm's 5 yr previous patent matching the citing patent's primary class
    6. Firm's 5 yr post patent matching the citing patent's primary class

In [107]:
for c in ['sim_docvecs_citing_asg_pc', 'sim_docvecs_citing_asg_cited_pc', 'sim_docvecs_citing_asg']:
    n1 = c3[['{0}_prior'.format(c), '{0}_post'.format(c), 'sim_docvecs']].dropna(how="any")
    display(n1.describe())
    print(sp.stats.ttest_ind(n1['{0}_prior'.format(c)], n1['{0}_post'.format(c)], equal_var = False))

,sim_docvecs_citing_asg_pc_prior,sim_docvecs_citing_asg_pc_post,sim_docvecs
count,1917.000000,1917.000000,1917.000000
mean,0.229258,0.234184,0.286055
std,0.125986,0.105972,0.151124
min,-0.163840,-0.070021,-0.422890
25%,0.149656,0.164750,0.185882
50%,0.225098,0.224147,0.281829
75%,0.302504,0.304338,0.382818
max,0.739014,0.616279,0.818367


Ttest_indResult(statistic=-1.3101515294604518, pvalue=0.1902254108531758)


,sim_docvecs_citing_asg_cited_pc_prior,sim_docvecs_citing_asg_cited_pc_post,sim_docvecs
count,1593.000000,1593.000000,1593.000000
mean,0.240144,0.243399,0.291386
std,0.122984,0.105119,0.149494
min,-0.104040,-0.094700,-0.422890
25%,0.164609,0.175223,0.188478
50%,0.230390,0.236063,0.286726
75%,0.306695,0.317031,0.388502
max,0.739014,0.641987,0.818367


Ttest_indResult(statistic=-0.8029639843979239, pvalue=0.42205693762299434)


,sim_docvecs_citing_asg_prior,sim_docvecs_citing_asg_post,sim_docvecs
count,5310.000000,5310.000000,5310.000000
mean,0.165243,0.180560,0.274551
std,0.118259,0.100331,0.144230
min,-0.252563,-0.171422,-0.422890
25%,0.088756,0.113474,0.175774
50%,0.158698,0.172586,0.272349
75%,0.234778,0.237309,0.367689
max,0.739014,0.691721,0.818367


Ttest_indResult(statistic=-7.197036305589004, pvalue=6.580663756750083e-13)
